### Import Statements

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show, ColumnDataSource, output_notebook, save
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues8
from bokeh.embed import components

from ipywidgets import interact


In [2]:
import warnings

warnings.simplefilter('ignore')

### Reading Data from CSV

In [3]:
#Get data from stored csv files

# Hindi GEC channels with start plus
hindi_gec_df = pd.read_csv('brand_grps_hindi_tg_r.csv')
hindi_gec_df.fillna('STAR Plus', inplace = True)
hindi_gec_df = hindi_gec_df.round(2)

# Movie channels
movies_df = pd.read_csv('brand_grps_movies_tg_r.csv')
movies_df.fillna('Movies OK', inplace = True)
movies_df = movies_df.round(2)

In [4]:
brand_list = hindi_gec_df['brand'].unique().tolist()

In [5]:
movies_df['channel'].unique()

array(['Movies OK', '&pictures', 'Zee Anmol Cinema', 'Zee Cinema',
       'Colors Cineplex', 'UTV Movies', 'B4U Movies'], dtype=object)

### Setting popular regions and targets for graph selection

In [6]:
region_list= ['HSM (EXCL. WB) (U)', 'MEGA CITIES', 'HSM (EXCL. NORTH EAST) (U)', 'MUM DEL BANG','ALL INDIA (U)','HSM (U+R)']

In [7]:
target_list= ['CS 15-40 AB','CS F 22-50 AB','CS 22-40 AB', 'CS F 22-50 ABC']

### Filtering dataset based on inputs from dropdowns

In [8]:
# Update all


def create_dataset(chan_type, target, brand,region):
    if chan_type == "Hindi GEC": 
        df = hindi_gec_df
        brand_list = hindi_gec_df['brand'].unique().tolist()


    elif chan_type == "Movie channel":
        df = movies_df
        brand_list = movies_df['brand'].unique().tolist()
        

    df = df[df['target']==target]
    df = df[df['regions']== region]
    df = df[df['brand']== brand]

    
    source = ColumnDataSource(df)
    return source


### Updating graph based on new filtered data

In [9]:
def update_plot(chan_type, target, brand, region):

    # setting bokeh output to show in notebook
    output_notebook()
    
    #getting converted columnsource dataset from function
    source = create_dataset(chan_type, target, brand, region)
    
    #creating list of channels based on type
    channel_list = source.data['channel'].tolist()
    
    #setting properties of the graph
    p = figure(
    y_range = channel_list,
    title = "{}|{}| {}| {}".format(chan_type, target, region, brand) ,
    x_axis_label = 'GRP',
    y_axis_label = 'Channel Names',
    tools = 'box_select, pan, zoom_in, zoom_out, reset')
    
    
    # plotting a horizontal bar from the dataset
    p.hbar(y='channel', right = 'grp', left=0, fill_alpha = 0.9, height =0.4, source = source, 
           fill_color = factor_cmap('channel', palette = Blues8, factors= channel_list))
    
    #setting legend properties
    p.legend.orientation = 'vertical'
    p.legend.location = 'bottom_right'
    p.legend.label_text_font_size = "8px"
    
    # adding hover option to show grp, ad length and weighted rating of selected channel
    hover = HoverTool()
    hover.tooltips = """

        <div>
            <h3>@channel</h3>
            <div><strong>GRP: @grp</strong></div>
            <div><strong>Total Length of ad: @tot_ad_len</strong></div>
            <div><strong>Weighted Rating: @wgt_rat</strong></div>
        </div>
    """
    p.add_tools(hover)
    
    #showing the graph
    show(p)

### Adding interactive options for graph

In [10]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()
interact(update_plot, chan_type=["Hindi GEC", "Movie channel"], target = target_list, brand = brand_list, region = region_list)

Loading BokehJS ...

interactive(children=(Dropdown(description='chan_type', options=('Hindi GEC', 'Movie channel'), value='Hindi G…

<function __main__.update_plot(chan_type, target, brand, region)>

## All Brands where STAR GRP is less than 50% of the highest GRP

In [11]:
star_channel_brands = pd.read_csv('channel_wise_star_grp.csv')
# star_channel_brands.head()
# star_channel_brands['channel'].unique()

In [12]:
star_brands = pd.read_csv('star_brands.csv')
non_star_brands = pd.read_csv('non_star_brands.csv')


In [13]:
non_star_channel_brands = pd.read_csv('channel_wise_non_star_grp.csv')
# non_star_channel_brands.head()
# non_star_channel_brands['channel'].unique()

In [14]:
#df[df['grp'] == df.groupby(['brand'])['grp'].transform(max)]

In [15]:
import numpy as np
import matplotlib.pyplot as plt

df = pd.concat([star_brands, non_star_brands])
df_default = pd.concat([non_star_channel_brands, star_channel_brands])

# Creates a data set based on user input and plots the graph
def plot_brands(star_plus, movies_ok, zee_tv, GRP_brand, STAR_Utsav,STAR_Gold,and_pictures, utv_action, sony_sab, colors, b4u_movies, big_magic, Sony_entertainment,Total_ad_length, brand, target, region):
    df = pd.concat([star_brands, non_star_brands])
    
    #Creating a list of channels based on input from user
    channel_list = []
    if star_plus==True:
        channel_list.append('STAR Plus')
    if movies_ok==True:
        channel_list.append('Movies OK')
    if zee_tv==True:
        channel_list.append('Zee TV')
    if STAR_Utsav==True:
        channel_list.append('STAR Utsav')
    if STAR_Gold==True:
        channel_list.append('STAR Gold')
    if and_pictures==True:
        channel_list.append('&pictures')
    if utv_action==True:
        channel_list.append('UTV Action')
    if sony_sab==True:
        channel_list.append('SONY SAB')
    if colors==True:
        channel_list.append('Colors')
    if b4u_movies==True:
        channel_list.append('B4U Movies')
    if big_magic==True:
        channel_list.append('Big Magic')
    if Sony_entertainment==True:
        channel_list.append('Sony Entertainment Television')
        
    
    #Creating a dataset with the channels from the list
    df1 = df[df['brand']==brand]
    df1 = df1[df1.channel.isin(channel_list)]
    target_list = df1['target'].unique().tolist()
    df1 = df1[df1['target']==target]
    region_list = df1['regions'].unique().tolist()
    df1 = df1[df1['regions']==region]


    # Plots the graph
    t= df1['channel']
    data1=df1['grp']
    data2 = df1['tot_ad_len']
    fig, ax1 = plt.subplots()
    fig.set_size_inches(25, 9)
    color = 'tab:red'
    ax1.set_xlabel('Channels')
    ax1.set_ylabel('GRP of Brands', color=color)
    if GRP_brand == True:
        ax1.plot(t, data1, color=color)
        ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  
    
    color = 'tab:blue'
    ax2.set_ylabel('Length of advertise in (s)', color=color) 
    
    if Total_ad_length==True:
        ax2.plot(t, data2, color=color)
        ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    plt.title(brand)
    plt.legend('brand')
    plt.show()

In [16]:
#df[df['grp'] == df.groupby(['channel'])['grp'].transform(max)] --3505.728183

brand_list_less = []

for b in non_star_channel_brands['brand'].unique():
    temp = non_star_channel_brands[non_star_channel_brands['brand']==b]
    grp  = temp['grp'].max()
    a = star_channel_brands[star_channel_brands['brand']==b]
    if len(a)==0:
            rn=1
    else:
            grp_star = a['grp'].max()
            if grp_star/grp < 0.5:
                brand_list_less.append(b)

In [17]:
interact(plot_brands,star_plus= True, movies_ok=True, zee_tv=True,STAR_Utsav=True,STAR_Gold = True,and_pictures=True, utv_action=False, sony_sab=True, colors=True, b4u_movies=True, big_magic=False, Sony_entertainment=True, GRP_brand=True, Total_ad_length=True, brand=brand_list_less, target=target_list, region=region_list )

interactive(children=(Checkbox(value=True, description='star_plus'), Checkbox(value=True, description='movies_…

<function __main__.plot_brands(star_plus, movies_ok, zee_tv, GRP_brand, STAR_Utsav, STAR_Gold, and_pictures, utv_action, sony_sab, colors, b4u_movies, big_magic, Sony_entertainment, Total_ad_length, brand, target, region)>

In [18]:
from ipywidgets import Checkbox, VBox, Layout, interactive
items = ['star_plus', 'movies_ok', 'zee_tv', 'GRP_brand', 'STAR_Utsav','STAR_Gold','and_pictures', 'utv_action', 'sony_sab', 'colors', 'b4u_movies', 'big_magic', 'Sony_entertainment']
checkbox_items = [Checkbox(description=name) for name in items]
container_layout = Layout(overflow_y='auto', display='block', height='240',border='1px solid black', width='300px')
VBox(children=checkbox_items, layout=container_layout)

#interact(plot_brands, star_plus=True, movies_ok=True)

In [19]:

widget=interactive(plot_brands,star_plus= True, movies_ok=True, zee_tv=True,STAR_Utsav=True,STAR_Gold = True,and_pictures=True, utv_action=False, sony_sab=True, colors=True, b4u_movies=True, big_magic=False, Sony_entertainment=True, GRP_brand=True, Total_ad_length=True, brand=brand_list_less, target=target_list, region=region_list )


controls = VBox(children=checkbox_items, layout=container_layout)
output = widget.children[-1]
display(VBox([controls, output]))

In [20]:

interact(plot_brands_2, channel_type=['Hindi GEC', 'Movie Channel'] GRP_brand=True, Total_ad_length=True, brand=brand_list_less, target=target_list, region=region_list )




SyntaxError: invalid syntax (<ipython-input-20-fef238247138>, line 2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df = pd.concat([star_brands, non_star_brands])
df_default = pd.concat([non_star_channel_brands, star_channel_brands])

# Creates a data set based on user input and plots the graph
def plot_brands_2(channel_type, Total_ad_length, brand, target, region):
    df = pd.concat([star_brands, non_star_brands])
    
    #Creating a list of channels based on input from user
    #channel_list = []
    
        
    if channel_type == 'Hindi GEC':
        channel_list =  hindi_gec_df['channel'].unique().tolist()
    else:
        channel_list = movies_df['channel'].unique().tolist()
    #Creating a dataset with the channels from the list
    df1 = df[df['brand']==brand]
    df1 = df1[df1.channel.isin(channel_list)]
    target_list = df1['target'].unique().tolist()
    df1 = df1[df1['target']==target]
    region_list = df1['regions'].unique().tolist()
    df1 = df1[df1['regions']==region]


    # Plots the graph
    t= df1['channel']
    data1=df1['grp']
    data2 = df1['tot_ad_len']
    fig, ax1 = plt.subplots()
    fig.set_size_inches(25, 9)
    color = 'tab:red'
    ax1.set_xlabel('Channels')
    ax1.set_ylabel('GRP of Brands', color=color)
    if GRP_brand == True:
        ax1.plot(t, data1, color=color)
        ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  
    
    color = 'tab:blue'
    ax2.set_ylabel('Length of advertise in (s)', color=color) 
    
    if Total_ad_length==True:
        ax2.plot(t, data2, color=color)
        ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    plt.title(brand)
    plt.legend('brand')
    plt.show()

In [ ]:
interact(plot_brands,star_plus= True, movies_ok=True, zee_tv=True,STAR_Utsav=True,STAR_Gold = True,and_pictures=True, utv_action=False, sony_sab=True, colors=True, b4u_movies=True, big_magic=False, Sony_entertainment=True, GRP_brand=True, Total_ad_length=True, brand=brand_list_less, target=target_list, region=region_list )